### Min Cost Objective

$$ \min \sum_{t=1}^{7}Labor\_Cost\_Doctor * D_t + Labor\_Cost\_Nurse * N_t + C\_N\_Mask * N\_Mask_t + C\_S\_Mask * S\_Mask_t + C\_Vent * Vent_t + C\_Glove * Glove_t + C\_Gown * Gown_t + C\_Eye\_Pro * Eye\_Pro_t + C\_Flu * Flu_t + C\_Staff\_Gown * Staff\_Gown_t$$
   s.t.  
   $$ 4\_D\_Req * D_t + 4\_N\_Req * N_t \ge 4\_Pat_t$$
   $$ 3\_D\_Req * D_t + 3\_N\_Req * N_t \ge 3\_Pat_t$$
   $$ 2\_D\_Req * D_t + 2\_N\_Req * N_t \ge 2\_Pat_t$$
   $$ 1\_N\_Req * N_t \ge 1\_Pat_t $$
   $$ D_t = D_{t-1} + On\_D_t - Off\_D_t $$
   $$ N_t = N_{t-1} + On\_N_t - Off\_N_t $$
   $$Adm\_Pat_t = 4\_Pat_t + 3\_Pat_t + 2\_Pat_t $$
   $$Staff_t = D_t + N_t $$
   $$Tot\_Pat_t = 4\_Pat_t + 3\_Pat_t + 2\_Pat_t + 1\_Pat_t $$
   $$ Adm\_Pat_t \le Num\_Beds $$
   $$ Gown_{t-1} + Gown\_Order_t = Adm\_Pat_t + Gown_t $$
   $$ Eye\_Pro_{t-1} + Eye\_Pro\_Order_t = Staff_t + Eye\_Pro_t $$
   $$ N\_Mask_{t-1} + N\_Mask\_Order_t = Staff_t + N\_Mask_t $$
   $$ Staff\_Gown_{t-1} + Staff\_Gown\_Order_t = Staff_t + Staff\_Gown_t $$
   $$ S\_Mask_{t-1} + S\_Mask\_Order_t = Tot\_Pat_t + S\_Mask_t $$
   $$ Flu_{t-1} + Flu\_Order_t = Req\_Flu * Adm\_Pat_t + Flu_t $$
   $$ Glove_{t-1} + Glove\_Order_t = Req\_Glove * Staff_t + Glove_t$$
   $$ Vent_{t-1} + Vent\_Order_t = 4\_Pat_t + 3\_Pat_t + Vent_t $$
   $$ Gown_0 = Int\_Gown $$
   $$ Eye\_Pro_0 = Int\_Eye\_Pro $$
   $$ N\_Mask_0 = Int\_N\_Mask $$
   $$ Staff\_Gown_0 = Int\_Staff\_Gown $$
   $$ S\_Mask_0 = Int\_S\_Mask $$
   $$ Flu_0 = Int\_Flu $$
   $$ Glove_0 = Int\_Glove $$
   $$ Gown_t, Vent_t, Eye\_Pro_t, N\_Mask_t, Staff\_Gown_t, S\_Mask_t, Flu_t, Glove_t, D_t, N_t, Gown\_Order_t, Eye\_Pro\_Order_t, N\_Mask\_Order_t, Staff\_Gown\_Order_t, S\_Mask\_Order_t, Flu\_Order_t, Glove\_Order_t, Vent\_Order_t \ge 0 $$

### Patient Treatment Objective

* Create Treatment Path  

* Include time for each step and capacities for each step  

* Create logical constraints for different patient severities


In [2]:
using Random, StatsBase, NamedArrays


# Creating an array of new cases each day based on the statisical data
new_cases = [rand(6:16) for i in 1:1, j in 1:7] 

# Finding the number of Covid cases that required hospitalization from
# CDC data 
hospitalized = round.(Int, new_cases*0.1642) 

# Patients with severity 1 are those who don't need hospitalization
pat_1 = new_cases - hospitalized 

# Initialize categories of hospitalized patients and probabilities
items = ["2_Pat", "3_Pat", "4_Pat"]
weights = [0.6232, 0.206, 0.1707]

# Distribute new cases among our categories
distr = [sample(items, Weights(weights), i) for i in hospitalized]

# Distribute current cases among our categories
prior_pats = sample(items, Weights(weights), 41);
c = countmap(prior_pats)

# Initialize patient category arrays
pat_2 = zeros(1,7)
pat_3 = zeros(1,7)
pat_4 = zeros(1,7)

# Separate patients from new hospitalizations into category arrays
for i in 1:length(distr)
    for j in distr[i]
        if j == "4_Pat"
            pat_4[i] += 1
        end
        if j == "3_Pat"
            pat_3[i] += 1
        end
        if j == "2_Pat"
            pat_2[i] += 1
        end
    end
end

# Compute total daily patients for severity 3
count_3 = c["3_Pat"]
for i in 1:length(pat_3)
    if pat_3[i] == 0
        pat_3[i] = count_3
    else
        count_3 = count_3 + pat_3[i] 
        pat_3[i] = count_3
    end
end

# Compute total daily patients for severity 4
count_4 = c["4_Pat"]
for i in 1:length(pat_4)
    if pat_4[i] == 0
        pat_4[i] = count_4
    else
        count_4 = count_4 + pat_4[i] 
        pat_4[i] = count_4
    end
end

# Compute total daily patients for severity 2
count_2 = c["2_Pat"]
for i in 1:length(pat_2)
    if pat_2[i] == 0
        pat_2[i] = count_2
    else
        count_2 = count_2 + pat_2[i] 
        pat_2[i] = count_2
    end
end

# Combine categories into full demand schedule
days = [1, 2, 3, 4, 5, 6, 7]
sev = [1, 2, 3, 4]
Patients = [pat_1; pat_2; pat_3; pat_4]
demand = NamedArray(Patients, (sev, days), ("Severity Level", "Day"))

4×7 Named Array{Float64,2}
Severity Level ╲ Day │    1     2     3     4     5     6     7
─────────────────────┼─────────────────────────────────────────
1                    │  5.0   6.0   6.0   5.0   6.0   7.0   5.0
2                    │ 29.0  29.0  29.0  29.0  30.0  31.0  31.0
3                    │  9.0   9.0   9.0  10.0  10.0  10.0  10.0
4                    │  4.0   5.0   6.0   6.0   6.0   6.0   7.0

In [88]:
using JuMP, Gurobi


d_cost = 393.69          # Daily cost of each Doctor
n_cost = 180.47          # Daily cost of each Nurse
n95_mask_cost = 6.00     # Cost per N95 Mask
surg_mask_cost = 0.50    # Cost per Surgical Mask
gown_cost = 4.15         # Cost per Patient Gown
glove_cost = 0.25        # Cost per Pair of Gloves
shield_cost = 5.00       # Cost per Face Shield
vent_cost = 25000.00     # Cost per Ventilator
fluid_cost = 5.00        # Cost per Unit of Fluids
staff_gown_cost = 2.00   # Cost per Staff Gown


# Initial amounts of each consumable
gown_init = 3120
staff_gown_init = 3120
glove_init = 39000
n95_mask_init = 936
surg_mask_init = 1560
shield_init = 69
vent_init = 16
fluid_init = 339924

# Initial number of Doctors and Nurses
d_init = 80
n_init = 160


num_beds = 102 # Total number of beds in hospital

# Required consumables (per patient per day)
req_gown = 20
req_staff_gown = 20
req_glove = 250
req_n95_mask = 6
req_surg_mask = 10
req_fluid = 2179

# Staff needed per patient per day depending on patient severity
n_req_1 = 4
n_req_2 = 4
n_req_3 = 2
n_req_4 = 2
d_req_2 = 20
d_req_3 = 14
d_req_4 = 14

m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

# Gown Variables
@variable(m, gown[1:7] >= 0)
@variable(m, gown_order[1:7] >= 0)

# Staff Gown Variables
@variable(m, staff_gown[1:7] >= 0)
@variable(m, staff_gown_order[1:7] >= 0)

# Glove Variables
@variable(m, glove[1:7] >= 0)
@variable(m, glove_order[1:7] >= 0)

# N95 Mask Variables
@variable(m, n95_mask[1:7] >= 0)
@variable(m, n95_mask_order[1:7] >= 0)

# Surgical Mask Variables
@variable(m, surg_mask[1:7] >= 0)
@variable(m, surg_mask_order[1:7] >= 0)

# Face Shield Variables
@variable(m, shield[1:7] >= 0)
@variable(m, shield_order[1:7] >= 0)

# Ventilator Variables
@variable(m, vent[1:7] >= 0)
@variable(m, vent_order[1:7] >= 0)

# Fluid Variables
@variable(m, fluid[1:7] >= 0)
@variable(m, fluid_order[1:7] >= 0)

# Doctors and Nurses on shift
@variable(m, n[1:7] >= 0, Int)
@variable(m, d[1:7] >= 0, Int)

# Doctors and Nurses added to shift
@variable(m, n_on[1:7] >= 0)
@variable(m, d_on[1:7] >= 0)

# Doctors and Nurses taken off of shift
@variable(m, n_off[1:7] >= 0)
@variable(m, d_off[1:7] >= 0)

# Number of Admitted Patients must be less than the number of beds
@constraint(m, bed_constr[days in 1:7], (pat_2[days] + pat_3[days] + pat_4[days]) <= num_beds)

# Nurse Balancing
@constraint(m, nurse_bal_init, n[1] == n_init + n_on[1] - n_off[1])
@constraint(m, nurse_bal[days in 2:7], n[days] == n[days-1] + n_on[days] - n_off[days])

# Doctor Balancing
@constraint(m, doctor_bal_init, d[1] == d_init + d_on[1] - d_off[1])
@constraint(m, doctor_bal[days in 2:7], d[days] == d[days-1] + d_on[days] - d_off[days])


# Doctor and Nurse patient requirements

@constraint(m, sev_1_n_req[days in 1:7], pat_1[days] <= n_req_1 * n[days])  # Number of Nurses needed per Severity 1 Patient
@constraint(m, sev_2_n_req[days in 1:7], pat_2[days] <= n_req_2 * n[days])  # Number of Nurses needed per Severity 2 Patient
@constraint(m, sev_3_n_req[days in 1:7], pat_3[days] <= n_req_3 * n[days])  # Number of Nurses needed per Severity 3 Patient
@constraint(m, sev_4_n_req[days in 1:7], pat_4[days] <= n_req_4 * n[days])  # Number of Nurses needed per Severity 4 Patient
@constraint(m, sev_2_d_req[days in 1:7], pat_2[days] <= d_req_2 * d[days])  # Number of Doctors needed per Severity 2 Patient
@constraint(m, sev_3_d_req[days in 1:7], pat_3[days] <= d_req_3 * d[days])  # Number of Doctors needed per Severity 3 Patient
@constraint(m, sev_4_d_req[days in 1:7], pat_4[days] <= d_req_4 * d[days])  # Number of Doctors needed per Severity 4 Patient

# Expressions for Consumable use

@expression(m, gown_use, [req_gown * (pat_2[days] + pat_3[days] + pat_4[days]) for days in 1:7])
@expression(m, staff_gown_use, [req_staff_gown * (pat_1[days] + pat_2[days] + pat_3[days] + pat_4[days]) for days in 1:7])
@expression(m, glove_use, [req_glove * (pat_1[days] + pat_2[days] + pat_3[days] + pat_4[days]) for days in 1:7])
@expression(m, n95_mask_use, [req_n95_mask * (pat_1[days] + pat_2[days] + pat_3[days] + pat_4[days]) for days in 1:7])
@expression(m, surg_mask_use, [req_surg_mask * (pat_1[days] + pat_2[days] + pat_3[days] + pat_4[days]) for days in 1:7])
@expression(m, vent_use, [(pat_4[days] + pat_3[days]) for days in 1:7])
@expression(m, fluid_use, [req_fluid * (pat_2[days] + pat_3[days] + pat_4[days]) for days in 1:7])

# Consumable Inventory Flow Balance

# Balancing Patient Gowns
@constraint(m, gown_inv_bal_init, gown_init + gown_order[1] == gown_use[1] + gown[1])
@constraint(m, gown_inv_bal[days in 2:7], gown[days-1] + gown_order[days] == gown_use[days] + gown[days])

# Balancing Staff Gowns
@constraint(m, staff_gown_inv_bal_init, staff_gown_init + staff_gown_order[1] == staff_gown_use[1] + staff_gown[1])
@constraint(m, staff_gown_inv_bal[days in 2:7], staff_gown[days-1] + staff_gown_order[days] == staff_gown_use[days] + staff_gown[days])

# Balancing Gloves
@constraint(m, glove_inv_bal_init, glove_init + glove_order[1] == glove_use[1] + glove[1])
@constraint(m, glove_inv_bal[days in 2:7], glove[days-1] + glove_order[days] == glove_use[days] + glove[days])

# Balancing N95 Masks
@constraint(m, n95_mask_inv_bal_init, n95_mask_init + n95_mask_order[1] == n95_mask_use[1] + n95_mask[1])
@constraint(m, n95_mask_inv_bal[days in 2:7], n95_mask[days-1] + n95_mask_order[days] == n95_mask_use[days] + n95_mask[days])

# Balancing Surgical Masks
@constraint(m, surg_mask_inv_bal_init, surg_mask_init + surg_mask_order[1] == surg_mask_use[1] + surg_mask[1])
@constraint(m, surg_mask_inv_bal[days in 2:7], surg_mask[days-1] + surg_mask_order[days] == surg_mask_use[days] + surg_mask[days])

# Balancing Face Shields
@constraint(m, shield_inv_bal_init, shield_init + shield_order[1] == shield_use[1] + shield[1])
@constraint(m, shield_inv_bal[days in 2:7], shield[days-1] + shield_order[days] == shield_use[days] + shield[days])

# Balancing Ventilators
@constraint(m, vent_inv_bal_init, vent_init + vent_order[1] == vent_use[1] + vent[1])
@constraint(m, vent_inv_bal[days in 2:7], vent[days-1] + vent_order[days] == vent_use[days] + vent[days])

# Balancing Fluids
@constraint(m, fluid_inv_bal_init, fluid_init + fluid_order[1] == fluid_use[1] + fluid[1])
@constraint(m, fluid_inv_bal[days in 2:7], fluid[days-1] + fluid_order[days] == fluid_use[days] + fluid[days])


# Objective is to Minimize the Total Cost of Staff and Consumables used 
@objective(m, Min, d_cost * sum(d) + n_cost * sum(n) + n95_mask_cost * sum(n95_mask_use) + surg_mask_cost * sum(surg_mask_use) + gown_cost * sum(gown_use) + glove_cost * sum(glove_use) + shield_cost * sum(d + n) + vent_cost * sum(vent_use) + fluid_cost * sum(fluid_use) + staff_gown_cost * sum(staff_gown_use));


Academic license - for non-commercial use only


In [89]:
optimize!(m)
println("TOTAL COST")
println("="^40)
println("Cost: \$", round(objective_value(m), digits=2))
println()
println("DAILY STAFF REQUIREMENT")
println("="^40)
println("Doctors needed each day: ",Array(value.(d')))
println("Nurses needed each day: ", Array(value.(n')))
println()
println("DAILY CONSUMABLE USAGE")
println("="^40)
println("Gowns used each day: ", gown_use)
println("Staff Gowns used each day: ", staff_gown_use)
println("Gloves used each day: ", glove_use)
println("N95 Masks used each day: ", n95_mask_use)
println("Surgical Masks used each day: ", surg_mask_use)
println("Shield used each day: ", Array(value.(d'))+Array(value.(n')))
println("Ventilators used each day: ", vent_use)
println("Fluids used each day: ", fluid_use)
println()
println("DAILY CONSUMABLE ORDER")
println("="^40)
println("Gowns ordered: ", Array(value.(gown_order')))
println("Staff Gowns ordered: ", Array(value.(staff_gown_order')))
println("Gloves ordered: ", Array(value.(glove_order')))
println("N95 Masks ordered: ", Array(value.(n95_mask_order')))
println("Surgical Masks ordered: ", Array(value.(surg_mask_order')))
println("Shield ordered: ", Array(value.(shield_order')))
println("Ventilators ordered: ", Array(value.(vent_order')))
println("Fluids ordered: ", Array(value.(fluid_order')))
println()
println("DAILY CONSUMABLE INVENTORY")
println("="^40)
println("Gowns inventory: ", Array(value.(gown')))
println("Staff Gowns inventory: ", Array(value.(staff_gown')))
println("Gloves inventory: ", Array(value.(glove')))
println("N95 Masks inventory: ", Array(value.(n95_mask')))
println("Surgical Masks inventory: ", Array(value.(surg_mask')))
println("Shield inventory: ", Array(value.(shield')))
println("Ventilators inventory: ", Array(value.(vent')))
println("Fluids inventory: ", Array(value.(fluid')))

Academic license - for non-commercial use only
TOTAL COST
Cost: $6.19998048e6

DAILY STAFF REQUIREMENT
Doctors needed each day: [2.0 2.0 2.0 2.0 2.0 2.0 2.0]
Nurses needed each day: [8.0 8.0 8.0 8.0 8.0 8.0 8.0]

DAILY CONSUMABLE USAGE
Gowns used each day: [840.0, 860.0, 880.0, 900.0, 920.0, 940.0, 960.0]
Staff Gowns used each day: [940.0, 980.0, 1000.0, 1000.0, 1040.0, 1080.0, 1060.0]
Gloves used each day: [11750.0, 12250.0, 12500.0, 12500.0, 13000.0, 13500.0, 13250.0]
N95 Masks used each day: [282.0, 294.0, 300.0, 300.0, 312.0, 324.0, 318.0]
Surgical Masks used each day: [470.0, 490.0, 500.0, 500.0, 520.0, 540.0, 530.0]
Shield used each day: [10.0 10.0 10.0 10.0 10.0 10.0 10.0]
Ventilators used each day: [13.0, 14.0, 15.0, 16.0, 16.0, 16.0, 17.0]
Fluids used each day: [91518.0, 93697.0, 95876.0, 98055.0, 100234.0, 102413.0, 104592.0]

DAILY CONSUMABLE ORDER
Gowns ordered: [0.0 0.0 0.0 360.0 920.0 940.0 960.0]
Staff Gowns ordered: [0.0 0.0 0.0 800.0 1040.0 1080.0 1060.0]
Gloves ordere